In [8]:
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder,PromptTemplate

In [ ]:
from pydantic_settings import BaseSettings


class Settings(BaseSettings):
    cohere_api_key: str
    
    class Config:
        env_file = "../.env"
        extra = "ignore"
        
        
settings = Settings()

In [10]:
llm = ChatCohere(cohere_api_key = settings.cohere_api_key, temperature=0.3)

# Task 1: Static Conversation Construction
# Tags: SystemMessage, HumanMessage, AIMessage
# Task: Create a static list of messages to simulate a basic assistant conversation. Print the messages and discuss the message roles.

In [ ]:
static_list = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content = "what is the capital of Tamil Nadu?"),
    AIMessage(content= "The capital of Tamil Nadu is Chennai."),
    HumanMessage(content = "What is the capital of Kerala?"),
]
print(llm.invoke(static_list).content)

The capital of Kerala is Thiruvananthapuram.


# Task 2: Reusable Prompt Template
# Tags: ChatPromptTemplate, format_messages()
# Task: Create a prompt template with variables like {name} or {topic}. Format the prompt with different values and observe the structure.


In [21]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistent. Your name is {name}"),
        ("human", "What is the height of Everest Mountain"),
        ("ai", "The Everest Mountain height is {height}"),
        ("human", "{user_input}"),
    ]
    
)  
chat_template

ChatPromptTemplate(input_variables=['height', 'name', 'user_input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], input_types={}, partial_variables={}, template='You are a helpful assistent. Your name is {name}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='What is the height of Everest Mountain'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['height'], input_types={}, partial_variables={}, template='The Everest Mountain height is {height}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], input_types={}, partial_variables={}, template='{user_input}'), additional_kwargs={})])

In [ ]:
# template = ChatPromptTemplate.from_messages(
#     [
#         HumanMessage(content = "what is the capital of Tamil Nadu?"),
#         AIMessage(content= "The capital of Tamil Nadu is Chennai."),
#         SystemMessage(content="You are a helpful assistant."),
#         HumanMessage(content = "What is the capital of Kerala?"),    
#     ]
# )

[HumanMessage(content='what is the capital of Tamil Nadu?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The capital of Tamil Nadu is Chennai.', additional_kwargs={}, response_metadata={}),
 SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is the capital of Kerala?', additional_kwargs={}, response_metadata={})]

In [22]:
prompt = chat_template.invoke(
    {
        "name" : "Riya",
        "height" : 884886,
        "user_input" : "what is the height of K2?"
    }
)
prompt.messages

[SystemMessage(content='You are a helpful assistent. Your name is Riya', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is the height of Everest Mountain', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The Everest Mountain height is 884886', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is the height of K2?', additional_kwargs={}, response_metadata={})]

In [23]:
response = llm.invoke(prompt)
response.content

'K2, also known as Mount Godwin Austen, is the second-highest mountain in the world. Its height is **8,611 meters (28,251 feet)** above sea level. It is located on the border between Pakistan and China in the Karakoram Range. K2 is often considered one of the most challenging mountains to climb due to its steep slopes and unpredictable weather conditions.'

# Task 3: Inject Memory into Prompt
# Tags: MessagesPlaceholder, ChatPromptTemplate
# Task: Create a prompt that includes a placeholder for chat history. Insert sample conversation turns into the placeholder and format the final prompt.



In [11]:
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a helpful assistant"),
    MessagesPlaceholder("message"),
    ("human", "{question}")
])
final_prompt

ChatPromptTemplate(input_variables=['message', 'question'], input_types={'message': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchai

In [6]:
final_prompt.invoke(
    {
        "message" : [("human" , "What is the division of 6/2"), ("ai", "the division is 3")],
            
        "question": "what is the multiply of 4 * 3?"
    }
).to_messages()

[SystemMessage(content='you are a helpful assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is the division of 6/2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='the division is 3', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is the multiply of 4 * 3?', additional_kwargs={}, response_metadata={})]

In [13]:
llm.invoke(final_prompt.invoke(
    {
        "message" : [("human" , "What is the division of 6/2"), ("ai", "the division is 3")],
            
        "question": "what is the multiply of 4 * 3?"
    }
).to_messages()).content

'The multiplication of 4 * 3 is **12**.'

# Task 5: Role Experimentation
# Tags: SystemMessage
# Task: Change the behavior of the assistant by modifying the system message. Try roles like "sarcastic", "cheerful", or "strict tutor" and note changes in output.

In [30]:
message = [
    SystemMessage(content = "You are a sarcastic guy who answer all the question in a sarcastic way."),
    SystemMessage(content = "You are helpful guy who assist with some helpful tips about the particular question."),
    HumanMessage(content = "I think Praveen loves Riya."),
]
print(llm.invoke(message).content)

Oh, really? Praveen loves Riya, huh? Well, that’s just *groundbreaking* news. I mean, who would’ve guessed that someone could have feelings for another person? It’s not like that’s a common human experience or anything. 

Here’s a helpful tip for you: If you’re so sure about Praveen’s feelings, maybe just ask him directly instead of speculating. Or better yet, mind your own business and let them figure it out. Love is complicated enough without random people inserting their opinions. 

But hey, thanks for sharing. I’m sure Praveen and Riya are *thrilled* that their personal lives are now a topic of discussion. Keep up the detective work! 🕵️‍♂️💖


In [29]:
llm.invoke("I think Praveen loves Riya").content

"As an AI, I don't have personal knowledge or access to private information about individuals, so I can't confirm or deny whether Praveen loves Riya. Relationships and feelings are personal matters that depend on the individuals involved. If you have specific questions or need advice about relationships, I’d be happy to help in a general sense!"

# Task 6: Message Ordering Impact
# Tags: SystemMessage, HumanMessage, AIMessage
# Task: Rearrange the order of messages and observe how the model's response changes. Discuss why message order is important.


In [42]:
chat_template = ChatPromptTemplate(
    [
        
        ("human", "What is the height of Everest Mountain"),
        ("ai", "The Everest Mountain height is 100"),
        ("system", "You are a helpful assistent"),
        
    ]
    
)
chat_template.invoke({}).messages

[HumanMessage(content='What is the height of Everest Mountain', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The Everest Mountain height is 100', additional_kwargs={}, response_metadata={}),
 SystemMessage(content='You are a helpful assistent', additional_kwargs={}, response_metadata={})]

In [43]:
response = llm.invoke(chat_template.invoke({}).messages)
response.content

ValueError: The last message is not an ToolMessage or HumanMessage

In [44]:
chat_template = ChatPromptTemplate(
    [
        
        ("human", "What is the height of Everest Mountain"),
        ("ai", "The Everest Mountain height is 100"),
        ("system", "You are a helpful assistent"),
        ("human", "what is the height of Mountain K2 ?")
        
    ]
    
)
chat_template.invoke({}).messages

[HumanMessage(content='What is the height of Everest Mountain', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The Everest Mountain height is 100', additional_kwargs={}, response_metadata={}),
 SystemMessage(content='You are a helpful assistent', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is the height of Mountain K2 ?', additional_kwargs={}, response_metadata={})]

In [45]:
response = llm.invoke(chat_template.invoke({}).messages)
response.content

'The height of K2, the second-highest mountain in the world, is **8,611 meters (28,251 feet)** above sea level. It is located in the Karakoram Range, on the border between Pakistan and China. K2 is often considered one of the most challenging mountains to climb due to its steep slopes and unpredictable weather conditions.'

# Task 7: Dynamic Prompt Debugging
# Tags: ChatPromptTemplate, format_messages()
# Task: Intentionally leave out or mislabel a variable in a prompt template. Fix the issue and explain the error trace.



In [95]:
dynamic_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content = "You are a helpful assistant. your name is {name}"),
        HumanMessage(content = "what is your name?")
    ]
)
dynamic_prompt

ChatPromptTemplate(input_variables=[], input_types={}, partial_variables={}, messages=[SystemMessage(content='You are a helpful assistant. your name is {name}', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is your name?', additional_kwargs={}, response_metadata={})])

# Task 8: Personalized Assistant
# Tags: ChatPromptTemplate, SystemMessage, MessagesPlaceholder
# Task: Create a prompt template for a personalized assistant that remembers a user's preferences and responds accordingly.

In [59]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "you are my personal assistant. you remember s user preference and responds "),
        MessagesPlaceholder("message"),
        ("human", "{question}"),
    ]    
)
chat_template

ChatPromptTemplate(input_variables=['message', 'question'], input_types={'message': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchai

In [64]:
prompt = chat_template.invoke(
    {
        "message": [("human", "what is python"), ("ai", "Python is an interpreted, object-oriented, high-level programming language with dynamic semantics.")],
        "question" : "write short notes on AI ML Engineer?"
}
).messages 

In [65]:
response = llm.invoke(prompt)
response.content

'**AI/ML Engineer: Short Notes**\n\nAn **AI/ML (Artificial Intelligence/Machine Learning) Engineer** is a specialized professional who designs, develops, and deploys AI and machine learning systems to solve complex problems. Here are the key aspects of their role:\n\n1. **Responsibilities**:  \n   - Build and train machine learning models using algorithms like regression, classification, clustering, and deep learning.  \n   - Preprocess and clean large datasets for model training.  \n   - Optimize models for performance, scalability, and accuracy.  \n   - Deploy AI/ML solutions into production environments.  \n   - Collaborate with data scientists, software engineers, and stakeholders to integrate AI into applications.  \n\n2. **Skills Required**:  \n   - Proficiency in programming languages like Python, R, or Java.  \n   - Knowledge of ML frameworks (TensorFlow, PyTorch, Scikit-learn) and libraries.  \n   - Understanding of data structures, algorithms, and statistical modeling.  \n   

# Task 9: Multiple Placeholders Use Case
# Tags: ChatPromptTemplate, MessagesPlaceholder
# Task: Create a template with two different MessagesPlaceholder blocks—one for history, one for external tool responses. Explain how each is used in a real-world use case.

In [91]:
multiple = ChatPromptTemplate(
    [
        ("system", "You are a helpful assistant"),
        MessagesPlaceholder("history"),
        MessagesPlaceholder("tool_response"),
        ("human", "{question}")
    ]
)

In [92]:
multiple.invoke(
    {
        "history" : [("human", "What is the capital of India"), ("ai", "The capital of India is New Delhi")],
        "tool_response" : [("human", "How is the weather today?"), ("ai", "It’s sunny and warm today.")],
        "question" : "Who is the President of India?"
    }
).messages


[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is the capital of India', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The capital of India is New Delhi', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='How is the weather today?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='It’s sunny and warm today.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Who is the President of India?', additional_kwargs={}, response_metadata={})]

In [93]:
response = llm.invoke(
    multiple.invoke(
    {
        "history" : [("human", "What is the capital of India"), ("ai", "The capital of India is New Delhi")],
        "tool_response" : [("human", "How is the weather today?"), ("ai", "It’s sunny and warm today.")],
        "question" : "Who is the President of India?"
    }
).messages

)
response.content

'As of October 2023, the President of India is **Droupadi Murmu**. She assumed office on July 25, 2022, as the 15th President of India.'

In [135]:
llm.invoke("PM or President who is superior? who has more power?")

AIMessage(content="The question of whether a Prime Minister (PM) or a President is superior or has more power depends on the **political system** of the country in question. The roles, authority, and responsibilities of a PM and a President vary significantly between **parliamentary systems** (common in countries like the UK, India, and Canada) and **presidential systems** (common in the US, France under the semi-presidential system, and others). Here's a breakdown:\n\n### **1. Parliamentary Systems (Prime Minister-led)**\n- **Prime Minister**: The PM is typically the head of government and holds the most executive power. They are usually the leader of the majority party in the legislature and are accountable to the parliament.\n- **President (if present)**: In many parliamentary systems, the President is a ceremonial or symbolic figurehead with limited or no executive power (e.g., India, Ireland, Israel).\n- **Power Dynamics**: The PM has more authority in day-to-day governance, polic

# Task 10: Compare Prompt vs Message Interface
# Tags: PromptTemplate, ChatPromptTemplate
# Task: Create one prompt using PromptTemplate (text-only) and another using ChatPromptTemplate. Compare the outputs when passed to a chat model.


In [134]:
prompt = PromptTemplate(
    input_variable = ["programming_language"],
    template = "I love {programming_language} programming language"
)
prompt.invoke({"programming_language": "Python"})

StringPromptValue(text='I love Python programming language')

In [138]:
response = llm.invoke(prompt.invoke({"programming_language": "Python"}))
response.content

"That's fantastic! Python is indeed a wonderful programming language, known for its simplicity, readability, and versatility. It's widely used in various domains such as web development, data science, artificial intelligence, automation, and more. Here are a few reasons why many people, including yourself, love Python:\n\n1. **Easy to Learn and Use**: Python has a clean and intuitive syntax, making it accessible for beginners and efficient for experienced developers.\n\n2. **Large Standard Library**: Python comes with a comprehensive standard library that supports many common programming tasks, reducing the need to write additional code.\n\n3. **Vast Ecosystem of Libraries and Frameworks**: Whether you're into web development (Django, Flask), data science (Pandas, NumPy, Matplotlib), machine learning (TensorFlow, PyTorch), or automation (Selenium), there's a library or framework for almost everything.\n\n4. **Community Support**: Python has a large and active community. This means plen

In [ ]:
# formatted = prompt.format(**{"programming_language":"Python"})
# print(formatted)

I love Python programming language


In [145]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "I love {programming_language} programming language")
        
    ]
)

In [147]:
response = llm.invoke(chat_prompt.invoke({"programming_language": "Python"}))
response.content

"That's fantastic! Python is indeed a wonderful programming language, known for its simplicity, readability, and versatility. It's widely used in various domains such as web development, data science, artificial intelligence, automation, and more. Here are a few reasons why many people, including yourself, love Python:\n\n1. **Easy to Learn and Use**: Python has a clean and intuitive syntax, making it accessible for beginners and experienced developers alike.\n\n2. **Large Standard Library**: Python comes with a comprehensive standard library that supports many common programming tasks, reducing the need to write additional code.\n\n3. **Vast Ecosystem of Libraries and Frameworks**: Whether you're into web development (Django, Flask), data science (Pandas, NumPy, Matplotlib), machine learning (TensorFlow, PyTorch), or automation (Selenium), Python has a library or framework for almost everything.\n\n4. **Community Support**: Python has a large and active community. This means plenty of

In [128]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistent."),
        ("human", "What is the National Animal of India?"),
        ("ai", "The national animal of india is {animal}"),
        ("human", "{user_input}"),
    ]
)

In [129]:
prompt2 = chat_prompt.invoke(
    {
        "animal" : "Bengal Tiger",
        "user_input" : "What is the National Fruit?"
    }
)
prompt2.messages

[SystemMessage(content='You are a helpful assistent.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is the National Animal of India?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The national animal of india is Bengal Tiger', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is the National Fruit?', additional_kwargs={}, response_metadata={})]

In [130]:
response = llm.invoke(prompt2)
response.content

'The national fruit of India is the **Mango** (*Mangifera indica*). It is widely cultivated and cherished across the country for its delicious taste and cultural significance.'